In [1]:
import pandas as pd
import datetime

In [2]:
pd.options.display.max_columns=500
pd.options.display.max_colwidth = None
pd.options.display.max_rows = None

In [3]:
result = []
render = False

In [4]:
from dqn import Qnet
from torchsummary import summary

summary(Qnet(), (3,200,200))

C:\Users\sinny\anaconda3\envs\rl-learn\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 199, 199]             208
       BatchNorm2d-2         [-1, 16, 199, 199]              32
              ReLU-3         [-1, 16, 199, 199]               0
         MaxPool2d-4           [-1, 16, 99, 99]               0
            Conv2d-5           [-1, 32, 98, 98]           2,080
       BatchNorm2d-6           [-1, 32, 98, 98]              64
              ReLU-7           [-1, 32, 98, 98]               0
         MaxPool2d-8           [-1, 32, 32, 32]               0
            Linear-9                  [-1, 256]       8,388,864
             ReLU-10                  [-1, 256]               0
           Linear-11                    [-1, 9]           2,313
Total params: 8,393,561
Trainable params: 8,393,561
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forw

In [5]:
def print_result(df):
    
    df = df.copy()
    
    for i in range(10, len(df), 10):
        print(f"episode {i} average reward: {df[i-10:i].mean()['reward']} max reward: {df[i-10:i].max()['reward']}")

### Global Varaiable & Property

In [6]:
# global variable


agent = None
target_epsilon = None
savefile = None


# global property

render = False
show_action = False

### CODE

In [7]:
import gym

import random
import collections

import time
import threading

from dqn import *


def main():

    parameter = HyperParameter(
        batch_size=64,
        buffer_limit=50000,
        gamma=0.98,
        learning_rate=0.001
    )
    
    global agent
    global epsilon
    global render
    
    env = Environment()
    agent = DQNAgent(param=parameter, path=savefile)

    print_interval = 20
    score = 0.0

    for n_epi in range(100000):
        epsilon = max(0.01, 0.2 - 0.1 * (n_epi / 10000))
        
        if target_epsilon:
            epsilon = target_epsilon
        
        state = env.reset()
        done = False
        
        action_list = []
        episode_reward = 0
        
        while not done:
            action = agent.predict(state, epsilon)
            state_prime, reward, done, info = agent.step(env, state, action)

            state = state_prime
            
            score += reward   
            episode_reward += reward
            
            if render:
                env.render()
                import time
                time.sleep(0.01)
            
            if show_action:
                action_list.append(action)
        
        
        if show_action:
            print(action_list)
            action_list = []
        
        result.append({
            'reward': episode_reward,
            'epsilon': epsilon
        })
        
        if n_epi % print_interval == 0 and n_epi != 0:
            agent.update_network()

            print("n_episode :{}, score : {:.1f}, eps : {:.1f}%".format(
                n_epi, score / print_interval, epsilon * 100))
            score = 0.0

threading.Thread(target=main).start()            
#main()

In [23]:
df = pd.DataFrame(result)
df

,reward,epsilon
0,-318,0.20000
1,-442,0.19999
2,-522,0.19998
3,-290,0.19997
4,-442,0.19996
5,-340,0.19995
6,-324,0.19994
7,-373,0.19993
8,-503,0.19992
9,-330,0.19991


In [24]:
print_result(df)

episode 10 average reward: -388.4 max reward: -290.0
episode 20 average reward: -339.0 max reward: -243.0
episode 30 average reward: -447.3 max reward: -164.0
episode 40 average reward: -391.7 max reward: -257.0
episode 50 average reward: -490.8 max reward: -334.0
episode 60 average reward: -404.8 max reward: -288.0
episode 70 average reward: -450.0 max reward: -215.0
episode 80 average reward: -419.8 max reward: -268.0
episode 90 average reward: -335.4 max reward: -155.0
episode 100 average reward: -469.2 max reward: -295.0
episode 110 average reward: -374.4 max reward: -169.0
episode 120 average reward: -379.9 max reward: -207.0
episode 130 average reward: -447.2 max reward: -303.0
episode 140 average reward: -455.9 max reward: -246.0
episode 150 average reward: -413.7 max reward: -257.0
episode 160 average reward: -450.9 max reward: -302.0
episode 170 average reward: -370.8 max reward: -256.0
episode 180 average reward: -443.6 max reward: -286.0
episode 190 average reward: -415.4 ma

In [26]:
render = False

n_episode :1440, score : -383.4, eps : 18.6%
n_episode :1460, score : -438.9, eps : 18.5%
n_episode :1480, score : -433.3, eps : 18.5%
n_episode :1500, score : -432.2, eps : 18.5%
n_episode :1520, score : -395.6, eps : 18.5%
n_episode :1540, score : -372.0, eps : 18.5%
n_episode :1560, score : -424.4, eps : 18.4%
n_episode :1580, score : -374.5, eps : 18.4%
n_episode :1600, score : -441.6, eps : 18.4%
n_episode :1620, score : -418.9, eps : 18.4%
n_episode :1640, score : -459.6, eps : 18.4%
n_episode :1660, score : -384.5, eps : 18.3%
n_episode :1680, score : -400.1, eps : 18.3%
n_episode :1700, score : -406.0, eps : 18.3%
n_episode :1720, score : -424.2, eps : 18.3%
n_episode :1740, score : -448.9, eps : 18.3%
n_episode :1760, score : -404.9, eps : 18.2%
n_episode :1780, score : -389.2, eps : 18.2%
n_episode :1800, score : -406.2, eps : 18.2%
n_episode :1820, score : -443.2, eps : 18.2%
n_episode :1840, score : -385.1, eps : 18.2%
n_episode :1860, score : -409.5, eps : 18.1%
n_episode 

n_episode :5100, score : -380.5, eps : 14.9%
n_episode :5120, score : -394.8, eps : 14.9%
n_episode :5140, score : -394.4, eps : 14.9%
n_episode :5160, score : -464.6, eps : 14.8%
n_episode :5180, score : -431.1, eps : 14.8%
n_episode :5200, score : -419.2, eps : 14.8%
n_episode :5220, score : -412.4, eps : 14.8%
n_episode :5240, score : -478.4, eps : 14.8%
n_episode :5260, score : -394.0, eps : 14.7%
n_episode :5280, score : -433.1, eps : 14.7%
n_episode :5300, score : -345.6, eps : 14.7%
n_episode :5320, score : -425.8, eps : 14.7%
n_episode :5340, score : -410.8, eps : 14.7%
n_episode :5360, score : -365.1, eps : 14.6%
n_episode :5380, score : -398.6, eps : 14.6%
n_episode :5400, score : -418.6, eps : 14.6%
n_episode :5420, score : -389.6, eps : 14.6%
n_episode :5440, score : -406.4, eps : 14.6%
n_episode :5460, score : -386.7, eps : 14.5%
n_episode :5480, score : -462.4, eps : 14.5%
n_episode :5500, score : -374.9, eps : 14.5%
n_episode :5520, score : -449.9, eps : 14.5%
n_episode 

n_episode :8760, score : -334.3, eps : 11.2%
n_episode :8780, score : -399.1, eps : 11.2%
n_episode :8800, score : -410.4, eps : 11.2%
n_episode :8820, score : -433.0, eps : 11.2%
n_episode :8840, score : -463.5, eps : 11.2%
n_episode :8860, score : -411.8, eps : 11.1%
n_episode :8880, score : -447.6, eps : 11.1%
n_episode :8900, score : -428.4, eps : 11.1%
n_episode :8920, score : -411.0, eps : 11.1%
n_episode :8940, score : -418.0, eps : 11.1%
n_episode :8960, score : -482.1, eps : 11.0%
n_episode :8980, score : -418.9, eps : 11.0%
n_episode :9000, score : -458.8, eps : 11.0%
n_episode :9020, score : -481.1, eps : 11.0%
n_episode :9040, score : -397.4, eps : 11.0%
n_episode :9060, score : -432.2, eps : 10.9%
n_episode :9080, score : -449.4, eps : 10.9%
n_episode :9100, score : -364.9, eps : 10.9%
n_episode :9120, score : -415.1, eps : 10.9%
n_episode :9140, score : -414.2, eps : 10.9%
n_episode :9160, score : -460.9, eps : 10.8%
n_episode :9180, score : -401.2, eps : 10.8%
n_episode 

n_episode :12400, score : -419.4, eps : 7.6%
n_episode :12420, score : -374.4, eps : 7.6%
n_episode :12440, score : -424.8, eps : 7.6%
n_episode :12460, score : -430.5, eps : 7.5%
n_episode :12480, score : -422.8, eps : 7.5%
n_episode :12500, score : -409.3, eps : 7.5%
n_episode :12520, score : -433.8, eps : 7.5%
n_episode :12540, score : -468.1, eps : 7.5%
n_episode :12560, score : -438.1, eps : 7.4%
n_episode :12580, score : -410.2, eps : 7.4%
n_episode :12600, score : -406.9, eps : 7.4%
n_episode :12620, score : -505.1, eps : 7.4%
n_episode :12640, score : -451.9, eps : 7.4%
n_episode :12660, score : -393.9, eps : 7.3%
n_episode :12680, score : -472.6, eps : 7.3%
n_episode :12700, score : -426.8, eps : 7.3%
n_episode :12720, score : -457.3, eps : 7.3%
n_episode :12740, score : -398.6, eps : 7.3%
n_episode :12760, score : -461.9, eps : 7.2%
n_episode :12780, score : -424.7, eps : 7.2%
n_episode :12800, score : -451.9, eps : 7.2%
n_episode :12820, score : -464.1, eps : 7.2%
n_episode 

n_episode :16060, score : -478.9, eps : 3.9%
n_episode :16080, score : -479.1, eps : 3.9%
n_episode :16100, score : -401.4, eps : 3.9%
n_episode :16120, score : -514.6, eps : 3.9%
n_episode :16140, score : -522.0, eps : 3.9%


In [27]:
render = True

n_episode :16160, score : -466.5, eps : 3.8%
n_episode :16180, score : -433.1, eps : 3.8%
n_episode :16200, score : -483.9, eps : 3.8%
n_episode :16220, score : -438.8, eps : 3.8%
n_episode :16240, score : -424.8, eps : 3.8%
n_episode :16260, score : -465.2, eps : 3.7%
n_episode :16280, score : -502.0, eps : 3.7%
n_episode :16300, score : -458.4, eps : 3.7%
n_episode :16320, score : -474.3, eps : 3.7%
n_episode :16340, score : -488.7, eps : 3.7%
n_episode :16360, score : -444.8, eps : 3.6%
n_episode :16380, score : -487.6, eps : 3.6%
n_episode :16400, score : -422.6, eps : 3.6%
n_episode :16420, score : -439.5, eps : 3.6%
n_episode :16440, score : -444.1, eps : 3.6%
n_episode :16460, score : -487.8, eps : 3.5%
n_episode :16480, score : -445.2, eps : 3.5%
n_episode :16500, score : -473.2, eps : 3.5%
n_episode :16520, score : -467.8, eps : 3.5%
n_episode :16540, score : -477.3, eps : 3.5%
n_episode :16560, score : -440.4, eps : 3.4%
n_episode :16580, score : -482.8, eps : 3.4%
n_episode 

n_episode :19820, score : -498.6, eps : 1.0%
n_episode :19840, score : -470.2, eps : 1.0%
n_episode :19860, score : -456.1, eps : 1.0%
n_episode :19880, score : -511.6, eps : 1.0%
n_episode :19900, score : -479.2, eps : 1.0%
n_episode :19920, score : -473.1, eps : 1.0%
n_episode :19940, score : -480.1, eps : 1.0%
n_episode :19960, score : -512.8, eps : 1.0%
n_episode :19980, score : -494.1, eps : 1.0%
n_episode :20000, score : -506.4, eps : 1.0%
n_episode :20020, score : -484.1, eps : 1.0%
n_episode :20040, score : -491.6, eps : 1.0%
n_episode :20060, score : -543.5, eps : 1.0%
n_episode :20080, score : -482.6, eps : 1.0%
n_episode :20100, score : -509.2, eps : 1.0%
n_episode :20120, score : -514.1, eps : 1.0%
n_episode :20140, score : -470.9, eps : 1.0%
n_episode :20160, score : -514.2, eps : 1.0%
n_episode :20180, score : -499.0, eps : 1.0%
n_episode :20200, score : -464.9, eps : 1.0%
n_episode :20220, score : -479.4, eps : 1.0%
n_episode :20240, score : -497.6, eps : 1.0%
n_episode 

In [11]:
raise Exception

C:\Users\sinny\anaconda3\envs\rl-learn\lib\site-packages\gym\envs\atari\environment.py:269: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  "We strongly suggest supplying `render_mode` when "


Exception: 

In [ ]:
agent.save('20220518121700.pt')

[20220517 084300]
유령 먹는 점수가 eat 점수에 비해 커가지고 agent가 야비하게 아이템을 먹고 쳐 기다린다... 보상점수를 좀 줄여야 할 듯 하다.
결국엔 전체적인 게임 흐름이 아니라 독립되어 한 상태에서 보상 값읅 구하는거기때문에 유령 먹는점수를 하향하는게 매우 중요하다
왜 계속 구석에 가있나 햇네 ㅋㅋㅋ


ghost_reward 변수를 새롭게 정의후 200에서 0으로 수정 유령은 필수로 먹어야하는 요소가 아니며 살아남기 위한 요소라거 정의하도록 한다.

[20220518 083000] 한 목숨 당 death 제어하는거 제외 이유는 이전에 높은 점수를 얻었는데도 불구하고
해당 구조로 인해 쿠키를 많이 먹지 못한 시나리오에 비해 우선도가 떨어질수도 있기 때문\

빨리 죽을 수 있는 문제는 엡실론 값을 주어 죽는게 아닌 다른 방법을 취해 쿠키를 먹었을때 보상을 얻어 점수가 쌓이게 되닌깐 결국엔 그방향ㅇ
으로 갈것이라 예상됨

[20220518 121600] 왜 계속 팩맨이 구석탱이로 가나 생각해보니 데스에 대한 마이너스 값을 너무 크게 해서 팩맨이 소녀감성을 갖게되서
발생된 일이 아닌가 싶음... 타켓은 다음 상태를 의미하기 때문에 전체적인 관점에선 아닐 수 있으나 단일 시점에선 그럴 가능성이 있음 사망 값을 좀 낮추어볼까...? (Q-learning을 사용한다면 전체적인 시점이 아닌 짧은 시점에서 찾아보는 것은 어떨까 애초에 td(시간차 방식)으로 측정하므로

함수 근사치로 구한다는 것을 명심하자